# Spark SQL

In this lab, you will take your first steps using Spark's support for SQL and relational concepts.

## Objectives

1. Use the `SparkSession` to query data as relational tables using different APIs.
2. Create a user-defined SQL function and use it in a SQL query.

### Get a SparkSession

In order to work with Spark's SQL support, we need to first get our hands on a special context called `SparkSession`.  
The SparkSession class is the entry point into all functionality in Spark. 

> Note: as of Spark 2.0, SparkSession replaced SqlContext. However, we could still use SqlContext as it's being kept for backward compatibility.

We'll use SparkSession.builder to create a SparkSession. SparkSession.builder lets you define you application name and it also lets you set various parameters in the Spark config, although there is no need to do so for our simple example.
    

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Spark SQL").getOrCreate()

Now, we're ready to roll with Spark SQL.

###  Get some data

Let's get some data to work with by creating a simple RDD from a sample transaction data file called tx.csv found in this course's data directory.

Issue the following command:

In [2]:
lines = spark.sparkContext.textFile("/home/jovyan/Resources/tx.csv")

print(lines.count())

1265


### Map to transactions

The individual data items in the lines of text are comma separated, thefore we'll be using split function to parse the lines into individual items. Parsed items will be mapped into tuples representing transactions. Note, we're converting the amount into `float` to allow for numeric operations.

Let's take a sample of transactions to see what they look like.

In [3]:
txns = lines.map(lambda st: st.split(",")).map(lambda el: (el[0], el[1], float(el[2])))

txns.take(5)

[('2015-06-16',
  'POS Withdrawal - 75901 CORNER STORE 13        DRIPPING SPRITXUS - Card Ending In 7090',
  -66.21),
 ('2015-06-16',
  'Withdrawal - Online Banking Transfer To XXXXXXXXXX CK',
  -50.0),
 ('2015-06-16',
  'POS Withdrawal - 879042 HEB GAS #404           AUSTIN       TXUS - Card Ending In 7090',
  -45.49),
 ('2015-06-16',
  'POS Withdrawal - CVS 07607 7607--13510 W HIGHWAY AUSTIN       TXUS - Card Ending In 2000',
  -37.41),
 ('2015-06-16',
  'POS Withdrawal - DAIRY QUEEN # 436 777 CASINO AVENUE      THACKERVILLE OKUS - Card Ending In 2000',
  -4.56)]

### Get a `DataFrame`

The primary, table-like artifact in Spark SQL that represents data is called a `DataFrame`.  We'll be using an `RDD`s function called `toDF()` to transform it into a `DataFrame`.

> Note:  *Any* `RDD` can serve as the basis for a `DataFrame`.

A `DataFrame` can essentially be thought of as a table, with columns and rows.  Often, the schema of the table, that is, the names and types of the columns, can be inferred; this will happen when RDD uses elements of defined types. In our case the `RDD` transactions are represented by simple tuples so only the data types are inferred. Luckily we can define the column names by simply passing them to `toDF` function as an array. 

Let's get a `DataFrame` of our transaction data now.  Issue the command:


In [4]:
df = txns.toDF(["date", "desc", "amount"])

Take a look at the DataFrame's schema by issuing command:

In [5]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- amount: double (nullable = true)



We could also take a peek at the content of `DataFrame` as follows:

In [6]:
df.show()


+----------+--------------------+-------+
|      date|                desc| amount|
+----------+--------------------+-------+
|2015-06-16|POS Withdrawal - ...| -66.21|
|2015-06-16|Withdrawal - Onli...|  -50.0|
|2015-06-16|POS Withdrawal - ...| -45.49|
|2015-06-16|POS Withdrawal - ...| -37.41|
|2015-06-16|POS Withdrawal - ...|  -4.56|
|2015-06-16|POS Deposit - OMN...|   50.0|
|2015-06-16|POS Deposit - OMN...| 110.86|
|2015-06-17|Withdrawal - Onli...|-511.36|
|2015-06-17|Withdrawal - Onli...| -186.2|
|2015-06-17|POS Withdrawal - ...|-118.97|
|2015-06-17|Withdrawal - Onli...|  -85.0|
|2015-06-17|Withdrawal - Onli...|  -80.0|
|2015-06-17|Withdrawal - Onli...|  -9.17|
|2015-06-17|Deposit - Online ...|   75.0|
|2015-06-17|Deposit - Online ...|  480.0|
|2015-06-18|POS Withdrawal - ...|-750.31|
|2015-06-18|External Withdraw...| -350.0|
|2015-06-18|External Withdraw...|  -45.0|
|2015-06-18|Withdrawal - Onli...|  -30.0|
|2015-06-18|POS Withdrawal - ...| -27.81|
+----------+--------------------+-

### Running SQL Queries Programmatically

`DataFrame`s can be queried using SQL but they need to be registered as temporary views first.

Registering is done by `DataFrame`s `createOrReplaceTempView` function, which accepts a parameter with the view name.

SQL query is executed via `SparkSession.sql` function are it returns the result as another `DataFrame`.

That's all it takes in order for us to be able to query data in the Spark cluster via SQL!


In [7]:
df.createOrReplaceTempView("txns")

### Query some data with SQL

Let's do something simple to start out with:  count the number of rows in the table.  Issue the following command:

In [8]:
spark.sql("SELECT COUNT(*) AS count FROM txns").show()

+-----+
|count|
+-----+
| 1265|
+-----+



Nice!  You should start realizing the possibilities here.  Any program that speaks JDBC (or ODBC) can connect to a Spark cluster and start querying data, including COTS business intelligence tools (BIRT, etc) and custom programs.

Let's sink our teeth into Spark SQL's juicy SQL flesh by finding the top 10 credits in our transaction

In [9]:
spark.sql("SELECT * FROM txns WHERE amount > 0 ORDER BY amount DESC LIMIT 10").show()

+----------+--------------------+-------+
|      date|                desc| amount|
+----------+--------------------+-------+
|2015-12-15|Deposit - Online ...|2209.03|
|2015-08-05|Deposit - Shared ...| 1520.0|
|2015-12-15|Deposit - Online ...|1360.48|
|2015-12-07|Deposit - Online ...| 1000.0|
|2015-12-11|Deposit - Online ...|  925.0|
|2015-12-04|Deposit - Online ...|  800.0|
|2015-06-19|Deposit - Online ...| 750.31|
|2015-07-26|Deposit - Online ...|  650.0|
|2015-07-07|Deposit - Online ...|  500.0|
|2015-07-10|Deposit - Online ...|  500.0|
+----------+--------------------+-------+



It's a Thing of Beauty!  This data is stored in who-knows-what format in who-knows-what-kind-of-distributed-cluster, but we can query it like it's a table!  Let's keep going.

### Query some data with Spark SQL's query DSL

For those of you who groan at the sight of strings carrying program logic, this might just be for you.  Spark SQL includes a simple but effective query DSL that can mitigate errors in otherwise stringy SQL that can't be checked at compile-time.  Let's perform the same query, only this time, let's use the query DSL.  Issue the following command:

In [10]:
df.filter(df["amount"] > 0).orderBy("amount", ascending=False).limit(10).show()

+----------+--------------------+-------+
|      date|                desc| amount|
+----------+--------------------+-------+
|2015-12-15|Deposit - Online ...|2209.03|
|2015-08-05|Deposit - Shared ...| 1520.0|
|2015-12-15|Deposit - Online ...|1360.48|
|2015-12-07|Deposit - Online ...| 1000.0|
|2015-12-11|Deposit - Online ...|  925.0|
|2015-12-04|Deposit - Online ...|  800.0|
|2015-06-19|Deposit - Online ...| 750.31|
|2015-07-26|Deposit - Online ...|  650.0|
|2015-07-07|Deposit - Online ...|  500.0|
|2015-07-10|Deposit - Online ...|  500.0|
+----------+--------------------+-------+



We've greatly reduced the portion of our code that is contained in strings that are opaque to the compiler!

> Note:  Spark 1.6 will take type safety even farther with a new artifact called a `DataSet`, which allows your query DSL-based code to be *completely* type-safe.

Your output should be identical to your earlier query.  Let's keep going with the complementary query.  What are the biggest debits in our transaction data?


In [11]:
spark.sql("SELECT * FROM txns WHERE amount < 0 ORDER BY amount LIMIT 10").show()

+----------+--------------------+-------+
|      date|                desc| amount|
+----------+--------------------+-------+
|2015-12-14|External Withdraw...| -925.0|
|2015-12-07|ATM Withdrawal - ...| -803.0|
|2015-12-04|ATM Withdrawal - ...| -800.0|
|2015-06-18|POS Withdrawal - ...|-750.31|
|2015-06-17|Withdrawal - Onli...|-511.36|
|2015-07-28|External Withdraw...|-408.71|
|2015-06-27|ATM Withdrawal - ...|-405.99|
|2015-06-18|External Withdraw...| -350.0|
|2015-07-06|Withdrawal - Shar...| -350.0|
|2015-08-14|POS Withdrawal - ...|-335.12|
+----------+--------------------+-------+



And now, via the query DSL:

In [13]:
df.filter(df["amount"] < 0).orderBy("amount").limit(10).show()

+----------+--------------------+-------+
|      date|                desc| amount|
+----------+--------------------+-------+
|2015-12-14|External Withdraw...| -925.0|
|2015-12-07|ATM Withdrawal - ...| -803.0|
|2015-12-04|ATM Withdrawal - ...| -800.0|
|2015-06-18|POS Withdrawal - ...|-750.31|
|2015-06-17|Withdrawal - Onli...|-511.36|
|2015-07-28|External Withdraw...|-408.71|
|2015-06-27|ATM Withdrawal - ...|-405.99|
|2015-06-18|External Withdraw...| -350.0|
|2015-07-06|Withdrawal - Shar...| -350.0|
|2015-08-14|POS Withdrawal - ...|-335.12|
+----------+--------------------+-------+



Again, the results should be identical.

### Extend SQL with your own functions

Many SQL databases allow you to define your own functions and call them in your queries; these are called "user-defined functions", or UDFs.  Spark also supports these, and you can write them in Scala, Java, Python, or R.  Let's check it out.

Our custom function will be very simple:  it calculates the trimmed length of a string.  If you've ever done this in pure SQL, you've felt The Pain.  However, it's trivial in Python:

In [12]:
def strlen(s):
  return len(s)


Now you need to register it as a UDF in your `SparkSesssion`:

In [13]:
spark.udf.register("len", strlen)

<function __main__.strlen(s)>

Here, we've registered our function `strlen` in the `SparkSesssion` with the name `len`.  Now, let's use it to find extremely long transaction descriptions.

In [14]:
spark.sql("SELECT len(desc), desc FROM txns WHERE len(desc) >= 100 ORDER BY len(desc) DESC").show()

+---------+--------------------+
|len(desc)|                desc|
+---------+--------------------+
|      107|External Deposit ...|
|      107|External Deposit ...|
+---------+--------------------+



If you want to know the original descriptions use collect.

In [15]:
spark.sql("SELECT len(desc), desc FROM txns WHERE len(desc) >= 100 ORDER BY len(desc) DESC").collect()


[Row(len(desc)='107', desc='External Deposit - AMAZON.COM ID3LXQ2E - Marketplac  payments.amazon.com ID#U6X42QZ19K9LLOW U6X42QZ19K9LLOW'),
 Row(len(desc)='107', desc='External Deposit - AMAZON.COM IESC1WFF - Marketplac  payments.amazon.com ID#QNQK9Z43WLNNVIB QNQK9Z43WLNNVIB')]

There are only two transactions whose length exceeds 100 characters.  Notice that we've used our UDF in the `SELECT` clause, the `WHERE` clause, and the `ORDER BY` clause!  Nice, right?

### Create tables from other types of data

Spark SQL supports directly creating `DataFrame`s from several different formats natively, including JSON, Parquet, and even JDBC.  That's right:  you can create your own tables in the cluster from data in some *external* JDBC data source!  Let's have a look at reading some JSON data.

Let's read the sample transaction data that's been provided in JSON format.  Here's a snippet of what it looks like:

``` json
{ "date": "2015-06-16", "desc": "POS Withdrawal - 75901 CORNER STORE 13        DRIPPING SPRITXUS - Card Ending In 7090", "amount":-66.21 }
{ "date": "2015-06-16", "desc": "Withdrawal - Online Banking Transfer To XXXXXXXXXX CK", "amount":-50.0 }
{ "date": "2015-06-16", "desc": "POS Withdrawal - 879042 HEB GAS #404           AUSTIN       TXUS - Card Ending In 7090", "amount":-45.49 }
```

> Warning:  There's one thing to note about Spark SQL's JSON support:  each line must be a complete JSON object.  That's not usually the case with JSON.  Most blocks of JSON data are multiline.  The data above may be wrapping due to current margins, but each JSON object above is actually on one line.

To read the JSON data, issue the following commands:

In [16]:
jt = spark.read.json("/home/jovyan/Resources/tx.jsons")

jt.createOrReplaceTempView("jtx")

jt.printSchema()

root
 |-- amount: double (nullable = true)
 |-- date: string (nullable = true)
 |-- desc: string (nullable = true)



Hey, where did the `DataFrame` get that schema from?  Well, since the source is JSON, there is type information implicitly available!  In JSON, everything is a string that's wrapped in double-quotes.  Anything else has a definite type that can be inferred!  Here, we see that `amount` has been inferred to be a number, so the `DataFrame` is using `double`.

Now, let's issue the same query as before:  what are the biggest credits in the transaction data?

In [17]:
spark.sql("SELECT * FROM jtx WHERE amount > 0 ORDER BY amount DESC LIMIT 10").show()

+-------+----------+--------------------+
| amount|      date|                desc|
+-------+----------+--------------------+
|2209.03|2015-12-15|Deposit - Online ...|
| 1520.0|2015-08-05|Deposit - Shared ...|
|1360.48|2015-12-15|Deposit - Online ...|
| 1000.0|2015-12-07|Deposit - Online ...|
|  925.0|2015-12-11|Deposit - Online ...|
|  800.0|2015-12-04|Deposit - Online ...|
| 750.31|2015-06-19|Deposit - Online ...|
|  650.0|2015-07-26|Deposit - Online ...|
|  500.0|2015-07-10|Deposit - Online ...|
|  500.0|2015-07-07|Deposit - Online ...|
+-------+----------+--------------------+



Notice that this data is the same data as we got before.  The only difference is the column order.  Pretty cool, n'est-ce pas?

## Conclusion

In this lab, you've seen how you can use conventional SQL to query data stored in a Spark cluster, and that you can import data from many different sources, including other SQL data sources.  This is just the tip of the iceberg:  you can not only query data, but you can modify it, too, or even store `DataFrames` as permanent SQL tables in your cluster.  Further, Spark SQL even supports Hive & HiveQL via, you guessed it, `HiveContext`!  The sky's the limit now!

## Challenge

#### Find the amount totals for every month

#### Find the account balance at the end of each month





## Challenge solution

In [19]:
import pyspark.sql.functions as func

txn_month = df.select(df["date"][1:7].alias("month"), df["amount"])

# txn_month.createOrReplaceTempView("txn_month")
# txn_group = spark.sql("select month, sum(amount) as amount from txn_month group by month order by month")

txn_group = txn_month.groupBy("month").agg(func.sum("amount").alias("amount")).orderBy("month")
txn_group.show()

tot = 0
for mon in txn_group.collect():
   tot = tot + mon[1]
   print(mon[0], round(tot, 2))

+-------+-------------------+
|  month|             amount|
+-------+-------------------+
|2015-06|-2596.0400000000013|
|2015-07| -4679.479999999999|
|2015-08|-3163.5699999999993|
|2015-09|-4826.1100000000015|
|2015-10| -5754.909999999996|
|2015-11|           -5482.72|
|2015-12|  550.3000000000003|
+-------+-------------------+

2015-06 -2596.04
2015-07 -7275.52
2015-08 -10439.09
2015-09 -15265.2
2015-10 -21020.11
2015-11 -26502.83
2015-12 -25952.53
